In [2]:
import pyarrow.parquet as pq
import pandas as pd
import re
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('../dataset-4/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [3]:
df_elevated = df[df['username'].str.contains('NT AUTHORITY')]

In [4]:
import re
import pandas as pd

# Procesos que suelen ser "targets" de inyección
CRITICAL_PROCESSES = [
    "explorer.exe", "lsass.exe", "svchost.exe", "services.exe",
    "winlogon.exe", "csrss.exe", "smss.exe", "dllhost.exe"
]

# Procesos que suelen usarse como "injectors"
INJECTION_TOOLS = [
    "rundll32.exe", "regsvr32.exe", "mshta.exe", "powershell.exe",
    "cmd.exe", "wmic.exe", "cscript.exe", "wscript.exe"
]

# Patrones sospechosos en argumentos
SUSPICIOUS_ARG_PATTERNS = [
    r"\.dll", r"\.ocx", r" -enc", r" -encodedcommand",
    r"shellcode", r"inject", r"CreateRemoteThread", r"VirtualAlloc"
]

def hunt_process_injection(df, threshold=5):
    """
    Busca indicios de process injection en procesos críticos o herramientas de inyección.
    """
    df = df.copy()
    
    # --- 1) Critical processes con argumentos (inusual)
    mask_critical_with_args = (
        df['name'].str.lower().isin([p.lower() for p in CRITICAL_PROCESSES]) &
        df['arguments'].str.strip().ne("")   # con argumentos no vacíos
    )
    crit_susp = df[mask_critical_with_args]
    
    # --- 2) Procesos conocidos de inyección con payload sospechoso
    pattern = "|".join(SUSPICIOUS_ARG_PATTERNS)
    mask_injectors = (
        df['name'].str.lower().isin([p.lower() for p in INJECTION_TOOLS]) &
        df['arguments'].str.lower().str.contains(pattern, na=False)
    )
    injectors = df[mask_injectors]
    
    # --- 3) Rare critical processes (en < threshold hosts)
    host_counts = df[df['name'].str.lower().isin([p.lower() for p in CRITICAL_PROCESSES])] \
                    .groupby('name')['hostname'].nunique()
    rare_procs = host_counts[host_counts < threshold].index.tolist()
    rare_crit = df[df['name'].str.lower().isin([p.lower() for p in rare_procs])]
    
    # --- Combinar resultados
    results = pd.concat([crit_susp, injectors, rare_crit]).drop_duplicates()
    return results


In [8]:
df_1 = hunt_process_injection(df_elevated)

display(df_1)

,hostname,name,arguments,username,path,pid
0,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k networkservice -p -s Dnscache,NT AUTHORITY\SYSTEM,c:\windows\system32,316
1,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k localserviceandnoimpersonation -p -s SSDPSRV,NT AUTHORITY\SYSTEM,c:\windows\system32,1088
2,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k netsvcs -p -s ProfSvc,NT AUTHORITY\SYSTEM,c:\windows\system32,808
5,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k netsvcs -p,NT AUTHORITY\SYSTEM,c:\windows\system32,2512
9,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k localsystemnetworkrestricted -p -s NcbService,NT AUTHORITY\SYSTEM,c:\windows\system32,4532
...,...,...,...,...,...,...
17,WC0005TWT,rundll32.exe,"C:\Program Files\Windows NT\rundll32.exe C:\Windows\System32\winlogon.dll,main MTIzLjEyNC4xMjUuMTI2 NDQz",NT AUTHORITY\SYSTEM,C:\Program Files\Windows NT\rundll32.exe,3696
30,WL0005LTK,rundll32.exe,"C:\Program Files\Windows NT\rundll32.exe C:\Windows\System32\winlogon.dll,main MTIzLjEyNC4xMjUuMTI2 NDQz",NT AUTHORITY\SYSTEM,C:\Program Files\Windows NT\rundll32.exe,4924
41,WL0006GQA,rundll32.exe,"C:\Program Files\Windows NT\rundll32.exe C:\Windows\System32\winlogon.dll,main MTIzLjEyNC4xMjUuMTI2 NDQz",NT AUTHORITY\SYSTEM,C:\Program Files\Windows NT\rundll32.exe,2236
68,WN0001CYN,rundll32.exe,"C:\Program Files\Windows NT\rundll32.exe C:\Windows\System32\winlogon.dll,main MTIzLjEyNC4xMjUuMTI2 NDQz",NT AUTHORITY\SYSTEM,C:\Program Files\Windows NT\rundll32.exe,2100
